In [1]:
import importlib
import numpy as np
import pandas as pd
import utils

In [2]:
from keras.models import Model, load_model
from keras.layers import Input, Embedding, LSTM, Dense, Dropout, Flatten
from keras.models import Model

Using TensorFlow backend.


In [3]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/encoder-decoder/utils.py'>

In [4]:
emb = utils.load_embedding()

In [5]:
word2ind, ind2word = utils.load_index_word_map()

In [6]:
emb_dim = emb.shape[1]
vocab_size = len(word2ind)

In [7]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
embedding = Embedding(vocab_size, emb_dim, weights=[emb])
encoder_embedding_outputs = embedding(encoder_inputs)
encoder_lstm_dim = 100
_, state_h, state_c = LSTM(encoder_lstm_dim, return_state=True, return_sequences=True)(encoder_embedding_outputs)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
decoder_embedding_outputs = embedding(decoder_inputs)
decoder_lstm_dim = 100
decoder_lstm = LSTM(decoder_lstm_dim, return_sequences=True, return_state=True)
decoder_lstm_outputs, _, _ = decoder_lstm(decoder_embedding_outputs, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [8]:
# Compile & run training
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
encoder_input_data = pd.read_csv('encoder_input.csv').values
decoder_input_data = pd.read_csv('decoder_input.csv').values
decoder_target_data = pd.read_csv('decoder_target.csv').values
decoder_target_data = decoder_target_data.reshape((decoder_target_data.shape[0], decoder_target_data.shape[1], -1))

In [19]:
encoder_input_data.shape

(4462, 25)

In [20]:
decoder_input_data.shape

(4462, 26)

In [21]:
decoder_target_data.shape

(4462, 26, 1)

In [23]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None)          0                                            
____________________________________________________________________________________________________
input_1 (InputLayer)             (None, None)          0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, None, 50)      107050      input_1[0][0]                    
                                                                   input_2[0][0]                    
____________________________________________________________________________________________________
lstm_1 (LSTM)                    [(None, None, 100), ( 60400       embedding_1[0][0]       

In [22]:
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=2,
          epochs=1)

Epoch 1/1
4462/4462 [==============================] - 73s - loss: 1.5856 - acc: 0.7479    


In [6]:
encoder_model = Model(encoder_inputs, encoder_states)

In [7]:
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 50)          200       
_________________________________________________________________
lstm_1 (LSTM)                [(None, None, 100), (None 60400     
Total params: 60,600
Trainable params: 60,600
Non-trainable params: 0
_________________________________________________________________


In [8]:
decoder_state_input_h = Input(shape=(encoder_lstm_dim,))
decoder_state_input_c = Input(shape=(encoder_lstm_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_embedding_outputs, initial_state=decoder_states_inputs)

In [9]:
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_lstm_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    decoder_inputs = np.zeros((1, 1, emb_dim))
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = ind2word[str(sampled_token_index)]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence